In [562]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime

import os

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [563]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunas_covid.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210227,c7ba6f9424c59a543e8803495c1061db,PERSONAL DE SALUD,39.0,MASCULINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO
1,20210227,51ffbe83478f1e64da17fe2756a9594b,PERSONAL DE SALUD,37.0,FEMENINO,20210219,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
2,20210227,54048fe3b06c35f14350d7e81340bdbd,PERSONAL DE SALUD,44.0,MASCULINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
3,20210227,746ad64c513b5d53b987ed48a6b45f77,PERSONAL DE SALUD,54.0,FEMENINO,20210220,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,JOSE LEONARDO ORTIZ
4,20210227,80f0a41dd63af3b96d4a80bfab2eaa61,PERSONAL DE SALUD,35.0,FEMENINO,20210225,1,SINOPHARM,LAMBAYEQUE,LAMBAYEQUE,CHICLAYO,CHICLAYO


In [564]:
# Obtengo el conteo del número de valores en cada columnas
df.count()

FECHA_CORTE         265608
UUID                265608
GRUPO_RIESGO        265608
EDAD                265502
SEXO                264719
FECHA_VACUNACION    265608
DOSIS               265608
FABRICANTE          265608
DIRESA              265608
DEPARTAMENTO        265608
PROVINCIA           265608
DISTRITO            265608
dtype: int64

In [565]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LAMBAYEQUE' 'LIMA' 'AMAZONAS' 'LA LIBERTAD' 'CAJAMARCA' 'AYACUCHO'
 'APURIMAC' 'SAN MARTIN' 'PIURA' 'HUANUCO' 'LORETO' 'CALLAO' 'CUSCO' 'ICA'
 'UCAYALI' 'HUANCAVELICA' 'AREQUIPA' 'JUNIN' 'MADRE DE DIOS' 'PASCO'
 'PUNO' 'ANCASH' 'MOQUEGUA' 'TACNA' 'TUMBES']
['PERSONAL DE SALUD' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'PERSONAL DE SEGURIDAD' 'PERSONAL MILITAR Ó FF AA'
 'POLICIA NACIONAL DEL PERU' 'ESTUDIANTES DE CIENCIAS DE LA SALUD'
 'BRIGADISTAS']
['MASCULINO' 'FEMENINO' nan]


In [566]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [567]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Fenero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [568]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [569]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0         7       612      1060       619   
              MASCULINO        0         4       303       590       419   
ANCASH        FEMENINO         0         1       986      1772      1328   
              MASCULINO        0         6       393       860       683   
APURIMAC      FEMENINO         0         3       741      1380      1051   
              MASCULINO        0         3       275       670       662   
AREQUIPA      FEMENINO         0         4      1571      2935      2258   
              MASCULINO        0         4       532      1156       977   
AYACUCHO      FEMENINO         0         4       871      1567      1171   
              MASCULINO        0         4       308       750       750   
CAJAMARCA     FEMENINO         0         2      1243      2052      1320   
              MASCULINO        0         8       522      1075       741   
CALLAO        FEMENINO         0         5       941      2298      1930   
              MASCULINO        0         8       536      1075       961   
CUSCO         FEMENINO         0         9      1005      2508      1665   
              MASCULINO        0         9       414      1039       922   
HUANCAVELICA  FEMENINO         0         5       820      1176       685   
              MASCULINO        0         4       307       546       431   
HUANUCO       FEMENINO         0         4       924      1587      1044   
              MASCULINO        0        11       349       830       658   
ICA           FEMENINO         0         3      1166      1619      1152   
              MASCULINO        0         5       394       719       578   
JUNIN         FEMENINO         0         3      1807      2561      1544   
              MASCULINO        0         7       518      1056       727   
LA LIBERTAD   FEMENINO         0         4      1732      3184      2127   
              MASCULINO        0        10       800      1457      1198   
LAMBAYEQUE    FEMENINO         0         1      1143      2016      1522   
              MASCULINO        0         1       441       896       792   
LIMA          FEMENINO         0        39      7845     19744     15353   
              MASCULINO        0        61      3855      9214      7628   
LORETO        FEMENINO         0         2       516      1204       829   
              MASCULINO        0         6       376       768       520   
MADRE DE DIOS FEMENINO         0         4       174       355       259   
              MASCULINO        0         1        75       174       129   
MOQUEGUA      FEMENINO         0         3       418       735       627   
              MASCULINO        0         4       133       336       274   
PASCO         FEMENINO         0         1       428       612       449   
              MASCULINO        0         3       159       302       291   
PIURA         FEMENINO         0         7      1338      2464      2164   
              MASCULINO        0        11       607      1177      1079   
PUNO          FEMENINO         0         3       827      1611      1035   
              MASCULINO        0         0       273       695       599   
SAN MARTIN    FEMENINO         0         9       936      1669       836   
              MASCULINO        0        11       529      1019       581   
TACNA         FEMENINO         0         4       415       741       704   
              MASCULINO        0         6       222       331       322   
TUMBES        FEMENINO         0         0       314       512       428   
              MASCULINO        0         0       171       277       248   
UCAYALI       FEMENINO         0         3       562       893       649   
              MASCULINO        0         6       333       511       407   

EDAD                     [50, 60)  [60, 70)  [7

In [570]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             BRIG.  E. SALUD  FF.AA.  P. SALUD  P. SEG.  P.N.P.  \
DEPARTAMENTO  SEXO                                                            
AMAZONAS      FEMENINO     NaN       NaN     9.0    2629.0      4.0    12.0   
              MASCULINO    NaN       NaN    16.0    1594.0     24.0     5.0   
ANCASH        FEMENINO     NaN      37.0     NaN    5392.0      5.0     2.0   
              MASCULINO    NaN      25.0     5.0    2658.0     58.0     6.0   
APURIMAC      FEMENINO     NaN       NaN     NaN    3788.0     12.0     1.0   
              MASCULINO    NaN       NaN     NaN    2157.0     16.0     NaN   
AREQUIPA      FEMENINO     NaN       NaN     NaN    9861.0      5.0     NaN   
              MASCULINO    NaN       NaN     NaN    4038.0     52.0     NaN   
AYACUCHO      FEMENINO     NaN       1.0     NaN    4374.0     15.0     NaN   
              MASCULINO    NaN       NaN     NaN    2460.0     55.0     NaN   
CAJAMARCA     FEMENINO     NaN      12.0     2.0    5328.0     29.0    12.0   
              MASCULINO    NaN      10.0     NaN    2824.0     61.0     6.0   
CALLAO        FEMENINO     NaN       NaN     1.0    7402.0     17.0    29.0   
              MASCULINO    NaN       NaN     NaN    3679.0     92.0    23.0   
CUSCO         FEMENINO     NaN       NaN     NaN    6836.0      9.0     NaN   
              MASCULINO    NaN       NaN     1.0    3315.0     44.0     NaN   
HUANCAVELICA  FEMENINO     NaN       NaN     NaN    3073.0      NaN     1.0   
              MASCULINO    NaN       NaN     3.0    1583.0      2.0     NaN   
HUANUCO       FEMENINO     NaN       1.0     NaN    4311.0     24.0     2.0   
              MASCULINO    NaN       3.0     NaN    2340.0     47.0     NaN   
ICA           FEMENINO     NaN      30.0     1.0    5272.0     21.0    35.0   
              MASCULINO    NaN      16.0     NaN    2430.0     64.0    23.0   
JUNIN         FEMENINO     NaN      85.0     5.0    7273.0      9.0    17.0   
              MASCULINO    NaN      22.0    19.0    3003.0     56.0    13.0   
LA LIBERTAD   FEMENINO     NaN      12.0     NaN    8994.0     16.0     NaN   
              MASCULINO    NaN       5.0     NaN    4563.0    114.0     1.0   
LAMBAYEQUE    FEMENINO     NaN       NaN     3.0    6114.0      NaN     NaN   
              MASCULINO    NaN       NaN     NaN    2963.0      1.0     2.0   
LIMA          FEMENINO     NaN       4.0    18.0   60452.0    151.0   135.0   
              MASCULINO    1.0       NaN    11.0   29501.0    669.0    94.0   
LORETO        FEMENINO     NaN       1.0     1.0    3185.0      NaN     NaN   
              MASCULINO    NaN       1.0     1.0    2170.0     18.0     1.0   
MADRE DE DIOS FEMENINO     NaN       NaN     NaN     990.0      NaN     NaN   
              MASCULINO    NaN       NaN     NaN     517.0      2.0     NaN   
MOQUEGUA      FEMENINO     NaN       NaN     NaN    2269.0     14.0     1.0   
              MASCULINO    NaN       NaN     NaN    1092.0     40.0     NaN   
PASCO         FEMENINO     NaN       2.0     2.0    1828.0      7.0     NaN   
              MASCULINO    NaN       NaN     2.0     977.0     48.0     NaN   
PIURA         FEMENINO     NaN       6.0     4.0    7404.0     19.0     8.0   
              MASCULINO    1.0      14.0     2.0    3976.0     20.0     5.0   
PUNO          FEMENINO     NaN       1.0     NaN    4588.0     52.0     NaN   
              MASCULINO    NaN       NaN     NaN    2387.0     32.0     NaN   
SAN MARTIN    FEMENINO     NaN       NaN     NaN    4092.0     23.0     NaN   
              MASCULINO    NaN       NaN     NaN    2680.0     37.0     NaN   
TACNA         FEMENINO     NaN       NaN     NaN    2631.0      2.0     NaN   
              MASCULINO    NaN       NaN     NaN    1356.0      3.0     NaN   
TUMBES        FEMENINO     NaN       NaN     NaN    1508.0      6.0     NaN   
              MASCULINO    NaN       NaN     NaN     854.0     66.0     NaN   
UCAYALI       FEMENINO     NaN       NaN     NaN  

In [571]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    plt.clf()

    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{datetime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], BINS, orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{datetime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n'.format(filename, area))

    if show_plot:
        plt.show()

In [572]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [573]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [574]:
# Deparmanetos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)